# Federated Next Word Prediction with Director example
## Using low-level Python API

In [ ]:
# Install dependencies if not already installed
# !pip install tensorflow==2.5.1

# Connect to the Federation

In [ ]:
# Create a federation
from openfl.interface.interactive_api.federation import Federation

# please use the same identificator that was used in signed certificate
cliend_id = 'frontend'

# 1) Run with API layer - Director mTLS 
# If the user wants to enable mTLS their must provide CA root chain, and signed key pair to the federation interface
# cert_chain = 'cert/root_ca.crt'
# API_certificate = 'cert/frontend.crt'
# API_private_key = 'cert/frontend.key'

# federation = Federation(client_id='frontend', director_node_fqdn='localhost', director_port='50051', disable_tls=False,
#                        cert_chain=cert_chain, api_cert=API_certificate, api_private_key=API_private_key)

# --------------------------------------------------------------------------------------------------------------------

# 2) Run with TLS disabled (trusted environment)
# Federation can also determine local fqdn automatically
federation = Federation(client_id='frontend', director_node_fqdn='localhost', director_port='50051', tls=False)

In [3]:
shard_registry = federation.get_shard_registry()
shard_registry

{'env_one': {'shard_info': node_info {
    name: "env_one"
  }
  shard_description: "Dataset from The Whisperer in Darkness by H. P. Lovecraft"
  n_samples: 22492
  sample_shape: "3"
  target_shape: "1",
  'is_online': True,
  'is_experiment_running': False,
  'last_updated': '2021-08-25 16:30:02',
  'current_time': '2021-08-25 16:30:10',
  'valid_duration': seconds: 120}}

In [4]:
federation.target_shape

['1']

In [5]:
# First, request a dummy_shard_desc that holds information about the federated dataset 
dummy_shard_desc = federation.get_dummy_shard_descriptor(size=10)
sample, target = dummy_shard_desc[0]

## Creating a FL experiment using Interactive API

In [1]:
# from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface, FLExperiment
from openfl.interface.interactive_api.experiment import DataInterface

### Register dataset

In [2]:
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):

    def __init__(self, shard_descriptor, indices, batch_size):
        self.shard_descriptor = shard_descriptor
        self.batch_size = batch_size
        self.indices = indices
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]

        X, y = self.shard_descriptor[batch]
        return X, y

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))


# Now you can implement you data loaders using dummy_shard_desc
class NextWordSD(DataInterface):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @property
    def shard_descriptor(self):
        return self._shard_descriptor

    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor

        train = round(len(self) * 0.8)
        self.train_indeces = list(range(1, train))
        self.valid_indeces = list(range(train, len(self)))

    def __getitem__(self, index):
        return self.shard_descriptor[index]

    def __len__(self):
        return len(self.shard_descriptor)

    def get_train_loader(self):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        if self.kwargs['train_bs']:
            batch_size = self.kwargs['train_bs']
        else:
            batch_size = 64

        return DataGenerator(self.shard_descriptor, self.train_indeces, batch_size=batch_size)

    def get_valid_loader(self):
        """
        Output of this method will be provided to tasks without optimizer in contract
        """
        if self.kwargs['valid_bs']:
            batch_size = self.kwargs['valid_bs']
        else:
            batch_size = 512

        return DataGenerator(self.shard_descriptor, self.valid_indeces, batch_size=batch_size)

    def get_train_data_size(self):
        """
        Information for aggregation
        """
        return len(self.train_indeces)

    def get_valid_data_size(self):
        """
        Information for aggregation
        """
        return len(self.valid_indeces)


In [3]:
fed_dataset = NextWordSD(train_bs=64, valid_bs=512)

### Describe a model and optimizer

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam


class Model(tf.keras.Model):
    """Model definition."""

    def __init__(self, **kwargs):
        super().__init__()

        vocab_size = 48904
        self.lstm1 = LSTM(1000, return_sequences=True)
        self.lstm2 = LSTM(1000)
        self.d1 = Dense(1000, activation='tanh')
        self.d2 = Dense(vocab_size, activation='softmax')

    def call(self, x):
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.d1(x)
        x = self.d2(x)
        return x
    
    def get_config(self):
        config = dict({'name': self.name})
        config['layers'] = [
            {'class_name': layer.__class__.__name__,
             'config': layer.get_config()} 
            for layer in model._self_tracked_trackables
        ]

        return config

# Construct an instance of Model
model = Model()
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [17]:
# Copyright (C) 2020-2021 Intel Corporation
# SPDX-License-Identifier: Apache-2.0
"""Market shard descriptor."""

import re
from pathlib import Path

import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

from openfl.interface.interactive_api.shard_descriptor import ShardDescriptor


class NextWordShardDescriptor(ShardDescriptor):
    """"Data - any text."""

    def __init__(self, title: str = '', author: str = '') -> None:
        """Initialize NextWordShardDescriptor."""
        super().__init__()

        self.title = title
        self.author = author
        self.dataset_dir = list(Path.cwd().rglob(f'{title}.txt'))[0]
        self.data = self.load_data(self.dataset_dir)  # list of words
        self.X, self.y = self.get_sequences(self.data)

    def __len__(self):
        """Number of sequences."""
        return len(self.X)

    def __getitem__(self, index: int):
        """Return an item by the index."""
        return self.X[index], self.y[index]

    @property
    def sample_shape(self):
        """Return the sample shape info."""
        return ['3', '96']  # three vectors

    @property
    def target_shape(self):
        """Return the target shape info."""
        return ['48904']  # row at one-hot matrix with n = vocab_size

    @property
    def dataset_description(self) -> str:
        """Return the dataset description."""
        return f'Dataset from {self.title} by {self.author}'

    @staticmethod
    def load_data(path):
        """Load text file, return list of words."""
        file = open(path, 'r', encoding='utf8').read()
        data = re.findall(r'[a-z]+', file.lower())
        return data

    @staticmethod
    def get_sequences(data):
        """
        Transform words to sequences, for X transform to vectors as well.

        To make vocab, clean it and get keyed vectors:
            if not spacy.util.is_package('en_core_web_sm'):
                spacy.cli.download('en_core_web_sm')
            nlp = spacy.load('en_core_web_sm')
            clean_vocab_list = [word for word in nlp.vocab.strings if re.fullmatch(r'[a-z]+', word)]

            word_to_vector = pd.Series([], name='vector')
            for word in clean_vocab_list:
                word_to_vector[word] = nlp(word).vector
            word_to_vector.to_pickle('keyed_vectors.pkl')
        """
        # spacy en_core_web_sm vocab_size = 48904, vector_size = 96
        x_seq = []
        y_seq = []
        vectors = pd.read_pickle('keyed_vectors.pkl')
        for i in range(len(data) - 3):
            x = data[i:i + 3]  # make 3-grams
            y = data[i + 3]
            cur_x = [vectors[word] for word in x if word in vectors]
            if len(cur_x) == 3 and y in vectors:
                x_seq.append(cur_x)
                y_seq.append(vectors.index.get_loc(y))

        x_seq = np.array(x_seq)
        y_seq = to_categorical(y_seq, num_classes=vectors.shape[0])
        return x_seq, y_seq


#### Register model

### Define and register FL tasks

In [18]:
import tqdm

# TI = TaskInterface()

# @TI.register_fl_task(model='model', data_loader='train_loader', device='device')
def train(model, train_loader, device=''):  # https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
    if tf.test.is_gpu_available():
        device = tf.device('/gpu:0')
    else:
        device = tf.device('/cpu:0')

    with device:
#         train_loader = tqdm.tqdm(train_loader, desc='train')

        # Unpack the data
        x, y = train_loader
        y = tf.convert_to_tensor(y)
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = model.compiled_loss(y, y_pred, regularization_losses=model.losses)

        # Compute gradients
        trainable_vars = model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        model.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update metrics (includes the metric that tracks the loss)
        model.compiled_metrics.update_state(y, y_pred)
    # Return a dict mapping metric names to current value
    return {m.name: m.result() for m in model.metrics}


# @TI.register_fl_task(model='model', data_loader='val_loader', device='device')
def validate(model, val_loader, device=''):
    if tf.test.is_gpu_available():
        device = tf.device('/gpu:0')
    else:
        device = tf.device('/cpu:0')

    with device:
#         val_loader = tqdm.tqdm(val_loader, desc='validate')

        # Unpack the data
        x, y = val_loader
        y = tf.convert_to_tensor(y)
        # Compute predictions
        y_pred = model(x, training=False)
        # Updates the metrics tracking the loss
        model.compiled_loss(y, y_pred, regularization_losses=model.losses)
        # Update the metrics.
        model.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
    # Note that it will include the loss (tracked in self.metrics).
    return {m.name: m.result() for m in model.metrics}

In [ ]:
fed_dataset.shard_descriptor = NextWordShardDescriptor(title='metamorphosis')

for epoch in range(1000):
    train_loader = fed_dataset.get_train_loader()[epoch]
    metrics = train(model, train_loader)
    m = {key: value.numpy() for key, value in metrics.items()}
    print(f'Epoch {epoch} train {m}')
    
    val_loader = fed_dataset.get_valid_loader()[epoch]
    metrics = validate(model, val_loader)
    m = {key: value.numpy() for key, value in metrics.items()}
    print(f'Epoch {epoch} validate {m}')

Epoch 0 train {'loss': 10.798113, 'categorical_accuracy': 0.0}
Epoch 0 validate {'loss': 10.791787, 'categorical_accuracy': 0.015625}
Epoch 1 train {'loss': 10.79166, 'categorical_accuracy': 0.015625}
Epoch 1 validate {'loss': 10.785787, 'categorical_accuracy': 0.014756944}
Epoch 2 train {'loss': 10.78552, 'categorical_accuracy': 0.014802632}
Epoch 2 validate {'loss': 10.772667, 'categorical_accuracy': 0.011574074}
Epoch 3 train {'loss': 10.771158, 'categorical_accuracy': 0.01171875}
Epoch 3 validate {'loss': 10.745472, 'categorical_accuracy': 0.010850694}
Epoch 4 train {'loss': 10.742561, 'categorical_accuracy': 0.011402027}
Epoch 4 validate {'loss': 10.668691, 'categorical_accuracy': 0.011111111}
Epoch 5 train {'loss': 10.662229, 'categorical_accuracy': 0.011209239}
Epoch 5 validate {'loss': 10.528828, 'categorical_accuracy': 0.010416667}
Epoch 6 train {'loss': 10.509809, 'categorical_accuracy': 0.010511364}
Epoch 6 validate {'loss': 10.324217, 'categorical_accuracy': 0.011160715}
Ep

Epoch 56 validate {'loss': 9.152436, 'categorical_accuracy': 0.022540204}
Epoch 57 train {'loss': 9.137461, 'categorical_accuracy': 0.022735838}
Epoch 57 validate {'loss': 9.137461, 'categorical_accuracy': 0.022735838}
Epoch 58 train {'loss': 9.129047, 'categorical_accuracy': 0.02292835}
Epoch 58 validate {'loss': 9.129047, 'categorical_accuracy': 0.02292835}
Epoch 59 train {'loss': 9.110953, 'categorical_accuracy': 0.022746941}
Epoch 59 validate {'loss': 9.110953, 'categorical_accuracy': 0.022746941}
Epoch 60 train {'loss': 9.096177, 'categorical_accuracy': 0.02256838}
Epoch 60 validate {'loss': 9.096177, 'categorical_accuracy': 0.02256838}
Epoch 61 train {'loss': 9.087684, 'categorical_accuracy': 0.0225143}
Epoch 61 validate {'loss': 9.087684, 'categorical_accuracy': 0.0225143}
Epoch 62 train {'loss': 9.071761, 'categorical_accuracy': 0.022702573}
Epoch 62 validate {'loss': 9.071761, 'categorical_accuracy': 0.022702573}
Epoch 63 train {'loss': 9.056434, 'categorical_accuracy': 0.0227

## Time to start a federated learning experiment

In [12]:
# create an experimnet in federation
experiment_name = 'word_prediction_test_experiment'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name)

In [13]:
# If I use autoreload I got a pickling error

# The following command zips the workspace and python requirements to be transfered to collaborator nodes
fl_experiment.start(model_provider=MI, 
                    task_keeper=TI,
                    data_loader=fed_dataset,
                    rounds_to_train=2,
                    opt_treatment='RESET')

AttributeError: 'NoneType' object has no attribute 'accepted'

In [ ]:
# If user want to stop IPython session, then reconnect and check how experiment is going 
# fl_experiment.restore_experiment_state(MI)

fl_experiment.stream_metrics()

In [ ]:
# todo: add testing on metamorphosis